In [1]:
import lsdb

In [13]:
from distributed import Client

In [27]:
client = Client(n_workers=4, threads_per_worker=1)

/Users/smcmu/.pyenv/versions/lsdb/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53952 instead
  warnings.warn(


In [28]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:53952/status,
Dashboard: http://127.0.0.1:53952/status,Workers: 4
Total threads: 4,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53953,Workers: 4
Dashboard: http://127.0.0.1:53952/status,Total threads: 4
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:53967,Total threads: 1
Dashboard: http://127.0.0.1:53971/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:53956,


In [18]:
ztf = lsdb.read_hats('https://data.lsdb.io/hats/ztf_dr22/ztf_lc', margin_cache='https://data.lsdb.io/hats/ztf_dr22/ztf_lc_10arcs', columns=['objra', 'objdec', 'mag'])

In [19]:
ztf.columns

Index(['objra', 'objdec', 'mag'], dtype='object')

In [21]:
gaia = lsdb.read_hats('https://data.lsdb.io/hats/gaia_dr3/gaia', margin_cache='https://data.lsdb.io/hats/gaia_dr3/gaia_10arcs', columns=['ra', 'dec', 'phot_g_mean_mag'])

In [22]:
gaia.columns

Index(['ra', 'dec', 'phot_g_mean_mag'], dtype='object')

In [23]:
gaia.cone_search(10, 10, 1000).crossmatch_nested(ztf, n_neighbors=3)

,ra,dec,phot_g_mean_mag,ztf_lc
npartitions=1,,,,
"Order: 5, Pixel: 4885",double[pyarrow],double[pyarrow],double[pyarrow],"nested<objra: [float], objdec: [float], mag: [..."


In [29]:
res = gaia.cone_search(10, 10, 1000).crossmatch_nested(ztf, n_neighbors=3).compute()

In [33]:
gaia.cone_search(10, 10, 1000).crossmatch_nested(ztf, n_neighbors=3).to_hats("out/nested_out")

In [37]:
read_cat = lsdb.read_hats("out/nested_out")
read_cat

,ra,dec,phot_g_mean_mag,ztf_lc
npartitions=1,,,,
"Order: 5, Pixel: 4885",double[pyarrow],double[pyarrow],double[pyarrow],"nested<objra: [float], objdec: [float], mag: [..."


In [39]:
read_cat.dtypes["ztf_lc"]

nested<objra: [float], objdec: [float], mag: [list<element: float>], _dist_arcsec: [double]>

In [26]:
client.close()